In [88]:
#main
import pandas as pd
import numpy as np
import datetime as dt
import scipy as sc


In [104]:
df=pd.read_csv('EUROSTOXX50_2023_Dataset.csv', index_col=0)

In [116]:
df_ptf=df[824:1592] #we selected 3y from 20-03-2019 bckw.
df_ptf=df_ptf.loc[:,['ADSGn.DE','ALVG.DE','MUVGn.DE','OREP.PA']]
df_den=df[825:1593]
df_den=df_den.loc[:,['ADSGn.DE','ALVG.DE','MUVGn.DE','OREP.PA']]
df_ptf

,ADSGn.DE,ALVG.DE,MUVGn.DE,OREP.PA
Date,,,,
2016-03-21,100.10,145.10,180.60,156.40
2016-03-22,101.35,145.35,180.90,154.55
2016-03-23,103.15,145.50,180.25,158.00
2016-03-24,102.45,142.60,176.65,156.70
2016-03-29,102.40,143.10,177.50,157.60
...,...,...,...,...
2019-03-13,204.80,198.44,209.30,228.90
2019-03-14,211.60,199.40,211.50,232.60
2019-03-15,212.40,200.90,216.60,236.10


In [117]:
df_den

,ADSGn.DE,ALVG.DE,MUVGn.DE,OREP.PA
Date,,,,
2016-03-22,101.35,145.35,180.90,154.55
2016-03-23,103.15,145.50,180.25,158.00
2016-03-24,102.45,142.60,176.65,156.70
2016-03-29,102.40,143.10,177.50,157.60
2016-03-30,104.60,144.55,179.65,160.70
...,...,...,...,...
2019-03-14,211.60,199.40,211.50,232.60
2019-03-15,212.40,200.90,216.60,236.10
2019-03-18,207.40,200.20,216.60,235.20


In [113]:
alpha=0.95
notional=1e7
delta=1
n_asset=4
weights=np.ones((n_asset,1))/n_asset
np_den=df_den.to_numpy()
np_num=df_ptf.to_numpy()

In [114]:
returns=np.log(np_num/np_den)
returns

In [ ]:
#c=df.loc['2019-03-20',['ADSGn.DE','ALVG.DE','MUVGn.DE','OREP.PA']]
#r=df_ptf/c